In [1]:
import cv2 ### opencv
from deepface import DeepFace
import numpy as np
from mss import mss
import os

In [3]:
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

In [4]:
# Load Haar cascade for face detection.
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# Optionally, precompute your reference embeddings.
# For example, you might have a folder "reference_faces" with subfolders or images named with the label.
# You can run this once at startup to build your embeddings database.
# Alternatively, DeepFace.find() can search a folder directly.
db_path = r"C:\Users\User\Pictures\Face_recog_db"
def detect_and_recognize(screen_frame):
    # Convert frame to grayscale for face detection.
    gray_image = cv2.cvtColor(screen_frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5, minSize=(40, 40))
    
    for (x, y, w, h) in faces:
        # Extract face region of interest (ROI)
        face_roi = screen_frame[y:y+h, x:x+w]
        # Convert ROI from BGR to RGB (DeepFace expects RGB images)
        face_roi_rgb = face_roi ##cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
        
        try:
            # Use DeepFace.find() to compare the detected face against your reference database.
            # Setting enforce_detection=False to avoid errors if a face isn't perfectly aligned.
            
            
            results = DeepFace.find(img_path=face_roi_rgb, db_path=db_path, model_name= "Facenet", enforce_detection=False)

            ##result_ver = DeepFace.verify(face_roi_rgb, db_path, model_name="VGG-Face")

            ##Label = result_ver["verified"]


            
            
            if len(results) > 0 and len(results[0]) > 0:
                # Assuming results is a list of DataFrames (one per model)
                # Get the first match's identity, then extract the name from the file path.
                identity_path = results[0].iloc[0]["identity"]
                label = identity_path.split("/")[-1].split("\\")[-1]  # works for Linux/Mac and Windows paths
            else:
                label = "Unknown"
        except Exception as e:
            # If DeepFace fails to process, label as Unknown.
            label = "Unknown_from_exception"
            print("Recognition error:", e)
        
        # Draw rectangle and label on the frame.
        cv2.rectangle(screen_frame, (x, y), (x+w, y+h), (0, 255, 0), 4)
        cv2.putText(screen_frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return screen_frame

# Capture the screen feed from the second monitor.
with mss() as sct:
    monitor = sct.monitors[2]
    while True:
        # Grab the screen image.
        img = np.array(sct.grab(monitor))
        # Convert image from BGRA to BGR.
        screen_frame = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        
        # Detect faces and perform recognition.
        processed_frame = detect_and_recognize(screen_frame)
        
        cv2.imshow("My Face Detection & Recognition Project", processed_frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cv2.destroyAllWindows()

25-03-03 21:26:04 - Searching [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[ 58  41  33]
  [ 35  16   8]
  [ 31  12   3]
  ...
  [ 22   5   2]
  [ 22   5   2]
  [ 22   5   1]]

 [[ 25   8   2]
  [ 25   8   2]
  [ 26   8   1]
  ...
  [ 24   7   1]
  [ 24   7   1]
  [ 23   7   1]]

 [[ 23   7   1]
  [ 23   7   1]
  [ 24   7   1]
  ...
  [ 24   8   1]
  [ 25   8   1]
  [ 24   8   1]]] in 6 length datastore
25-03-03 21:26:09 - find function duration 5.478174924850464 seconds
25-03-03 21:26:09 - Searching [[[243 229 222]
  [242 228 222]
  [241 228 222]
  ...
  [248 232 228]
  [247 233 228]
  [246 233 228]]

 [[243 229 224]
  [241 228 223]
  [241 228 222]
  ...
  [247 232 228]
  [247 233 228]
  [247 233 229]]

 [[243 230 